In [1]:
import pandas as pd 
import math
from sklearn.cross_validation import KFold   #For K-fold cross validation
import numpy as np
from sklearn import preprocessing, cross_validation,    svm
from sklearn.linear_model import LinearRegression
from  datetime import datetime
import matplotlib.pyplot as plt 
from matplotlib import style
import time
from sklearn import metrics
style.use('ggplot')
from sklearn.linear_model import LinearRegression
%matplotlib inline

In [6]:
df = pd.read_csv("data.csv")
df

,Line,Speaker,Text,Date,Party,Location,URL
0,1,Woodruff,"Good evening, and thank you. We are happy to w...",2/11/16,Democratic,"Milwaukee, Wisconsin",http://www.presidency.ucsb.edu/ws/index.php?pi...
1,2,Ifill,We want to also extend our warm thanks to Milw...,2/11/16,Democratic,"Milwaukee, Wisconsin",http://www.presidency.ucsb.edu/ws/index.php?pi...
2,3,AUDIENCE,(APPLAUSE),2/11/16,Democratic,"Milwaukee, Wisconsin",http://www.presidency.ucsb.edu/ws/index.php?pi...
3,4,Woodruff,"Welcome, Senator, great to see you. And former...",2/11/16,Democratic,"Milwaukee, Wisconsin",http://www.presidency.ucsb.edu/ws/index.php?pi...
4,5,AUDIENCE,(APPLAUSE),2/11/16,Democratic,"Milwaukee, Wisconsin",http://www.presidency.ucsb.edu/ws/index.php?pi...
5,6,Woodruff,Very good to be here with you.,2/11/16,Democratic,"Milwaukee, Wisconsin",http://www.presidency.ucsb.edu/ws/index.php?pi...
6,7,Clinton,Thank you.,2/11/16,Democratic,"Milwaukee, Wisconsin",http://www.presidency.ucsb.edu/ws/index.php?pi...
7,8,Ifill,Welcome to you both.,2/11/16,Democratic,"Milwaukee, Wisconsin",http://www.presidency.ucsb.edu/ws/index.php?pi...
8,9,Woodruff,"Now, a word about format. There will be two sh...",2/11/16,Democratic,"Milwaukee, Wisconsin",http://www.presidency.ucsb.edu/ws/index.php?pi...
9,10,Ifill,"With Iowa and New Hampshire behind us, we are ...",2/11/16,Democratic,"Milwaukee, Wisconsin",http://www.presidency.ucsb.edu/ws/index.php?pi...


In [7]:
df = df.drop('Date',1)
df = df.drop('URL',1)
df = df.drop('Line',1)

In [8]:
df[:5]

,Speaker,Text,Party,Location
0,Woodruff,"Good evening, and thank you. We are happy to w...",Democratic,"Milwaukee, Wisconsin"
1,Ifill,We want to also extend our warm thanks to Milw...,Democratic,"Milwaukee, Wisconsin"
2,AUDIENCE,(APPLAUSE),Democratic,"Milwaukee, Wisconsin"
3,Woodruff,"Welcome, Senator, great to see you. And former...",Democratic,"Milwaukee, Wisconsin"
4,AUDIENCE,(APPLAUSE),Democratic,"Milwaukee, Wisconsin"


In [21]:
doc_complete  = df['Text'].tolist()

In [23]:
import gensim
from gensim import corpora
from nltk.corpus import stopwords 
from nltk.stem.wordnet import WordNetLemmatizer
import string

stop = set(stopwords.words('english'))
exclude = set(string.punctuation) 
lemma = WordNetLemmatizer()

def clean(doc):
    stop_free = " ".join([i for i in doc.lower().split() if i not in stop])
    punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
    #normalized = " ".join(lemma.lemmatize(word) for word in punc_free.split())
    return punc_free

doc_clean = [clean(doc).split() for doc in doc_complete]    
#print(doc_clean)  

In [24]:
doc_clean

[['good',
  'evening',
  'thank',
  'you',
  'happy',
  'welcome',
  'milwaukee',
  'democratic',
  'debate',
  'especially',
  'pleased',
  'thank',
  'partners',
  'facebook',
  'helped',
  'us',
  'set',
  'vibrant',
  'conversation',
  'among',
  'voters',
  'undecided',
  'tonight',
  'youre',
  'going',
  'hear',
  'questions',
  'candidates',
  'follow',
  'along',
  'home',
  'pbs',
  'newshour',
  'page',
  'facebook',
  'also',
  'want',
  'thank',
  'hosts',
  'university',
  'wisconsin',
  'milwaukee',
  'whose',
  'campus',
  'meet',
  'beautiful',
  'helen',
  'bader',
  'concert',
  'hall'],
 ['want',
  'also',
  'extend',
  'warm',
  'thanks',
  'milwaukee',
  'public',
  'radio',
  'milwaukee',
  'public',
  'television',
  'well',
  'friends',
  'pbs',
  'member',
  'stations',
  'across',
  'country',
  'tuning',
  'tonight',
  'sixth',
  'time',
  'democrats',
  'met',
  'face',
  'face',
  'time',
  'learn',
  'presidents',
  'say',
  'want',
  'be',
  'know',
  'y

In [29]:
corpora.Dictionary(doc_clean)
from nltk.corpus import stopwords 
from nltk.stem.wordnet import WordNetLemmatizer
import string
dictionary = corpora.Dictionary(doc_clean)
doc_term_matrix = [dictionary.doc2bow(doc) for doc in doc_clean]

In [32]:
Lda = gensim.models.ldamodel.LdaModel
ldamodel = Lda(doc_term_matrix, num_topics= 10, id2word = dictionary, passes=80)
print(ldamodel.print_topics(num_topics= 8, num_words= 5))

[(6, u'0.022*isis + 0.018*us + 0.016*would + 0.014*iran + 0.013*war'), (1, u'0.035*crosstalk + 0.018*yes + 0.018*no + 0.015*commercial + 0.014*on'), (9, u'0.067*applause + 0.019*tax + 0.016*jobs + 0.014*percent + 0.012*people'), (3, u'0.026*trump + 0.025*donald + 0.021*clinton + 0.021*hillary + 0.014*mr'), (2, u'0.077*senator + 0.069*you + 0.061*governor + 0.056*thank + 0.019*secretary'), (8, u'0.014*wall + 0.014*laughter + 0.013*president + 0.012*street + 0.010*court'), (7, u'0.028*people + 0.012*country + 0.012*immigration + 0.011*law + 0.010*would'), (0, u'0.023*president + 0.017*america + 0.016*country + 0.014*need + 0.014*states')]


In [38]:
lda = gensim.models.ldamodel.LdaModel(corpus=doc_term_matrix, id2word=dictionary, num_topics= 8, update_every=0, passes=80)
#lda = gensim.models.ldamodel.LdaModel(corpus=doc_term_matrix, id2word=dictionary, num_topics=10, update_every=0, passes=20)
print(lda.print_topics(num_topics= 8, num_words= 6))

KeyboardInterrupt: 